from jupysec.rules import Rules

Rules().get_findings()

In [1]:
import requests
import json
import pandas as pd
import re
import nltk
import os

In [2]:
# importing `config.py` to access its variables, not working on windows
from config import github_token_value

In [3]:
	pd.options.display.max_columns = 250 #Changes the number of columns diplayed (default is 20)
	pd.options.display.max_rows = 250 #Changes the number of rows diplayed (default is 60)
	pd.options.display.max_colwidth = 250 #Changes the number of characters in a cell so that the contents don't get truncated (default is 50)

In [4]:
import requests

# Set the API endpoint URL 
url = "https://api.github.com/repos/{owner}/{repo}/issues"

# Set the owner and repo variables
owner = "Significant-Gravitas"
repo = "Auto-GPT"

# Set the authentication token (if required)
token = github_token_value

# Set the query parameters for the API request
params = {
    "state": "open",  # retrieve all issues, including closed ones
    "per_page": 100,  # retrieve up to 100 issues per page
    "page": 1  # start with the first page of results
}

In [5]:

issues = []

while True:
    response = requests.get(url.format(owner=owner, repo=repo), params=params, auth=(token, ""))

    if response.status_code != 200:
        print("Error retrieving issues from GitHub API")
        break

    page_issues = response.json()
    issues += page_issues

    if "Link" not in response.headers:
        break

    link_header = response.headers["Link"]
    next_url = None
    for link in link_header.split(","):
        if "rel=\"next\"" in link:
            next_url = link.split(";")[0][1:-1]
            break

    if next_url is None:
        break

    print(params)    
    params["page"] += 1

print(f"Retrieved {len(issues)} issues from GitHub")

# Print the response headers
print("Response Headers:")
for key, value in response.headers.items():
    print(f"{key}: {value}")
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

{'state': 'open', 'per_page': 100, 'page': 1}
{'state': 'open', 'per_page': 100, 'page': 2}
{'state': 'open', 'per_page': 100, 'page': 3}
{'state': 'open', 'per_page': 100, 'page': 4}
{'state': 'open', 'per_page': 100, 'page': 5}
{'state': 'open', 'per_page': 100, 'page': 6}
Retrieved 631 issues from GitHub
Response Headers:
Server: GitHub.com
Date: Tue, 02 May 2023 15:25:42 GMT
Content-Type: application/json; charset=utf-8
Transfer-Encoding: chunked
Cache-Control: public, max-age=60, s-maxage=60
Vary: Accept, Accept-Encoding, Accept, X-Requested-With
ETag: W/"9724bddafb351a4102676b754e13d6d44c4b83075005e2925037660fc5e15475"
X-GitHub-Media-Type: github.v3; format=json
Link: <https://api.github.com/repositories/614765452/issues?state=open&per_page=100&page=6>; rel="prev", <https://api.github.com/repositories/614765452/issues?state=open&per_page=100&page=1>; rel="first"
x-github-api-version-selected: 2022-11-28
X-RateLimit-Limit: 60
X-RateLimit-Remaining: 46
X-RateLimit-Reset: 1683044519

In [6]:
# create subdirectory if it does not exist
if not os.path.exists("subdata"):
    os.makedirs("subdata")

# loop over issues and save each one as a separate JSON file in the subdirectory
for i, issue in enumerate(issues):
    filename = f"subdata/issue-{i+1}.json"  # use the issue number as filename in subdirectory
    with open(filename, "w") as f:
        json.dump(issue, f)
    print(f"Exported JSON to {filename}")

Exported JSON to subdata/issue-1.json
Exported JSON to subdata/issue-2.json
Exported JSON to subdata/issue-3.json
Exported JSON to subdata/issue-4.json
Exported JSON to subdata/issue-5.json
Exported JSON to subdata/issue-6.json
Exported JSON to subdata/issue-7.json
Exported JSON to subdata/issue-8.json
Exported JSON to subdata/issue-9.json
Exported JSON to subdata/issue-10.json
Exported JSON to subdata/issue-11.json
Exported JSON to subdata/issue-12.json
Exported JSON to subdata/issue-13.json
Exported JSON to subdata/issue-14.json
Exported JSON to subdata/issue-15.json
Exported JSON to subdata/issue-16.json
Exported JSON to subdata/issue-17.json
Exported JSON to subdata/issue-18.json
Exported JSON to subdata/issue-19.json
Exported JSON to subdata/issue-20.json
Exported JSON to subdata/issue-21.json
Exported JSON to subdata/issue-22.json
Exported JSON to subdata/issue-23.json
Exported JSON to subdata/issue-24.json
Exported JSON to subdata/issue-25.json
Exported JSON to subdata/issue-26.

In [7]:
# Export the resulting JSON to a file
with open("github-issues.json", "w") as outfile:
    json.dump(issues, outfile)

print("Exported JSON to github-issues.json")

Exported JSON to github-issues.json
time: 62.5 ms (started: 2023-05-02 11:25:43 -04:00)


In [8]:
issues_df = pd.json_normalize(issues)

print(issues_df.head())

                                                                      url   
0  https://api.github.com/repos/Significant-Gravitas/Auto-GPT/issues/3695  \
1  https://api.github.com/repos/Significant-Gravitas/Auto-GPT/issues/3693   
2  https://api.github.com/repos/Significant-Gravitas/Auto-GPT/issues/3692   
3  https://api.github.com/repos/Significant-Gravitas/Auto-GPT/issues/3690   
4  https://api.github.com/repos/Significant-Gravitas/Auto-GPT/issues/3688   

                                               repository_url   
0  https://api.github.com/repos/Significant-Gravitas/Auto-GPT  \
1  https://api.github.com/repos/Significant-Gravitas/Auto-GPT   
2  https://api.github.com/repos/Significant-Gravitas/Auto-GPT   
3  https://api.github.com/repos/Significant-Gravitas/Auto-GPT   
4  https://api.github.com/repos/Significant-Gravitas/Auto-GPT   

                                                                             labels_url   
0  https://api.github.com/repos/Significant-Gravitas/Au

In [9]:
issues_df.size
print(len(issues_df.index))
issues_df.to_csv('issues_df.csv')

631
time: 74.6 ms (started: 2023-05-02 11:25:43 -04:00)


In [10]:
# iterating the columns
for col in issues_df.columns:
    print(col)

url
repository_url
labels_url
comments_url
events_url
html_url
id
node_id
number
title
labels
state
locked
assignee
assignees
milestone
comments
created_at
updated_at
closed_at
author_association
active_lock_reason
draft
body
timeline_url
performed_via_github_app
state_reason
user.login
user.id
user.node_id
user.avatar_url
user.gravatar_id
user.url
user.html_url
user.followers_url
user.following_url
user.gists_url
user.starred_url
user.subscriptions_url
user.organizations_url
user.repos_url
user.events_url
user.received_events_url
user.type
user.site_admin
pull_request.url
pull_request.html_url
pull_request.diff_url
pull_request.patch_url
pull_request.merged_at
reactions.url
reactions.total_count
reactions.+1
reactions.-1
reactions.laugh
reactions.hooray
reactions.confused
reactions.heart
reactions.rocket
reactions.eyes
assignee.login
assignee.id
assignee.node_id
assignee.avatar_url
assignee.gravatar_id
assignee.url
assignee.html_url
assignee.followers_url
assignee.following_url
assign

In [11]:
# copy dataframe
issues_title=issues_df.copy(deep=True)
issues_body=issues_df.copy(deep=True)


time: 12.2 ms (started: 2023-05-02 11:25:43 -04:00)


In [28]:
issues_body = issues_body[['number', 'body']]

time: 1.28 ms (started: 2023-05-02 11:27:02 -04:00)


In [29]:
print(issues_body)

     number   
0      3695  \
1      3693   
2      3692   
3      3690   
4      3688   
..      ...   
626      25   
627      21   
628      15   
629      11   
630       5   

                                                                                                                                                                                                                                                          body  
0    <!-- ⚠️ At the moment any non-essential commands are not being merged.\r\nIf you want to add non-essential commands to Auto-GPT, please create a plugin instead.\r\nWe are expecting to ship plugin support within the week (PR #757).\r\nResources:\r...  
1    ### ⚠️ Search for existing issues first ⚠️\n\n- [X] I have searched the existing issues, and there is no existing issue for my problem\n\n### Which Operating System are you using?\n\nMacOS\n\n### Which version of Auto-GPT are you using?\n\nStable...  
2    \r\n\r\n<!-- ⚠️ At the moment any non-essen

In [30]:
issues_body.reset_index(inplace=True)

time: 959 µs (started: 2023-05-02 11:27:02 -04:00)


In [31]:
print(issues_body)

     index  number   
0        0    3695  \
1        1    3693   
2        2    3692   
3        3    3690   
4        4    3688   
..     ...     ...   
626    626      25   
627    627      21   
628    628      15   
629    629      11   
630    630       5   

                                                                                                                                                                                                                                                          body  
0    <!-- ⚠️ At the moment any non-essential commands are not being merged.\r\nIf you want to add non-essential commands to Auto-GPT, please create a plugin instead.\r\nWe are expecting to ship plugin support within the week (PR #757).\r\nResources:\r...  
1    ### ⚠️ Search for existing issues first ⚠️\n\n- [X] I have searched the existing issues, and there is no existing issue for my problem\n\n### Which Operating System are you using?\n\nMacOS\n\n### Which version of Aut

In [32]:
issues_body.info(verbose = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631 entries, 0 to 630
Columns: 3 entries, index to body
dtypes: int64(2), object(1)
memory usage: 14.9+ KB
time: 4.98 ms (started: 2023-05-02 11:27:03 -04:00)


In [33]:
file_name = "issue_autogpt.txt"

time: 213 µs (started: 2023-05-02 11:27:03 -04:00)


In [34]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mark/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

time: 2.17 ms (started: 2023-05-02 11:27:03 -04:00)


In [35]:
# Open the text file
with open(file_name, 'r', encoding="utf-8") as f:

  # Read the file and tokenize it into sentences
  text = f.read()
  sentences = nltk.sent_tokenize(text)

  # Convert the sentences into an array
  sentences_array = []
  for sentence in sentences:
    sentence = sentence.strip()  # Remove whitespace at the beginning and end of the sentence
    if sentence:  # Check if the sentence is not empty
      print(sentence)
      sentences_array.append(sentence)

⚠️ Before you continue
Check out our backlog, roadmap and join our discord to discuss what's going on
If you need help, you can ask in the discussions section or in #tech-support
Throughly search the existing issues before creating a new one
⚠️ Search for existing issues first ⚠️
Please search the history to see if an issue already exists for the same problem.
I have searched the existing issues, and there is no existing issue for my problem *
Please provide a searchable summary of the issue in the title above ⬆️.
⚠️ SUPER-busy repo, please help the volunteer maintainers.
The less time we spend here, the more time we spend building AutoGPT.
Please help us help you:

Does it work on stable branch (https://github.com/Torantulino/Auto-GPT/tree/stable)?
Does it work on current master (https://github.com/Torantulino/Auto-GPT/tree/master)?
Search for existing issues, "add comment" is tidier than "new issue"
Ask on our Discord (https://discord.gg/autogpt)
Provide relevant info:
Provide commit

In [36]:
issues_body_clean=issues_body.copy(deep=True)

time: 1.25 ms (started: 2023-05-02 11:27:04 -04:00)


In [37]:
for sentence1 in sentences_array:
    issues_body_clean['body']=issues_body_clean['body'].str.replace(sentence1,'')

time: 53.6 ms (started: 2023-05-02 11:27:05 -04:00)


In [38]:
print(sentences_array[10])

Please select which version of Auto-GPT you were using when this issue occurred.
time: 454 µs (started: 2023-05-02 11:27:06 -04:00)


In [39]:
issues_body_clean.info(verbose = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631 entries, 0 to 630
Columns: 3 entries, index to body
dtypes: int64(2), object(1)
memory usage: 14.9+ KB
time: 8.47 ms (started: 2023-05-02 11:27:07 -04:00)


In [40]:
issues_body_clean

,index,number,body
0,0,3695,"<!-- ⚠️ At the moment any non-essential commands are not being merged.\r\nIf you want to add non-essential commands to Auto-GPT, please create a plugin instead.\r\nWe are expecting to ship plugin support within the week (PR #757).\r\nResources:\r..."
1,1,3693,"### ⚠️ Search for existing issues first ⚠️\n\n- [X] I have searched the existing issues, and there is no existing issue for my problem\n\n### \n\nMacOS\n\n### Which version of Auto-GPT are you using?\n\nStable (branch)\n\n### GPT-3 or GPT-4?\n\nG..."
2,2,3692,"\r\n\r\n<!-- ⚠️ At the moment any non-essential commands are not being merged.\r\nIf you want to add non-essential commands to Auto-GPT, please create a plugin instead.\r\nWe are expecting to ship plugin support within the week (PR #757).\r\nReso..."
3,3,3690,"Updating to encourage the user to build a virtual environment first.\r\n\r\n<!-- ⚠️ At the moment any non-essential commands are not being merged.\r\nIf you want to add non-essential commands to Auto-GPT, please create a plugin instead.\r\nWe are..."
4,4,3688,"### Background\r\n\r\nCurrently, only ALLOWLISTED_PLUGINS read from env.\r\n\r\nAs more plugins are created, you have to click ""n"" for each plugin you don't want to use - enabling DENYLISTED_PLUGINS will increase usability.\r\n\r\nThere is anothe..."
...,...,...,...
626,626,25,"You can modify the code to accept a config file as input, and read the Chosen_Model flag to select the appropriate AI model. Here's an example of how to achieve this:\r\n\r\nCreate a sample config file named config.ini:\r\n\r\n[AI]\r\nChosen_Mode..."
627,627,21,"It doesn't work?\r\n\r\n<img width=""1440"" alt=""image"" src=""https://user-images.githubusercontent.com/6225438/229346042-a8687d08-8c4d-4094-8da4-b2300fb51732.png"">\r\n"
628,628,15,"## Idea 💡\r\nThe **ULTIMATE** achievement for this project would be if Auto-GPT was able to recursively improve itself. That, after-all, is how AGI is predicted by many to come about. \r\n\r\n## Suggestion 👩‍💻\r\nAuto-GPT should be able to:\r\n\r..."
629,629,11,"_Context:_ Implement **reflection**, a technique that allows generating more coherent and natural texts using pre-trained language models.\r\n\r\n_Problem or idea:_ Reflection is based on two articles that propose different methods to incorpor..."


time: 16.9 ms (started: 2023-05-02 11:27:09 -04:00)


In [41]:
issues_body_clean_good=issues_body_clean.copy(deep=True)

time: 2.16 ms (started: 2023-05-02 11:27:11 -04:00)


In [57]:
issues_body_clean = issues_body_clean_good.copy(deep=True)

time: 508 µs (started: 2023-05-02 13:31:00 -04:00)


In [58]:
# convert all non-string columns to string
issues_body_clean = issues_body_clean.astype(str)

# define a regular expression to match non-alphanumeric and non-space characters
regex_unknown = re.compile(r"[^a-zA-Z0-9\s]")

# apply the regular expression to each element of the DataFrame
issues_body_clean = issues_body_clean.applymap(lambda s: regex_unknown.sub(" ", s))



time: 90 ms (started: 2023-05-02 13:31:01 -04:00)


In [59]:
issues_body_clean.to_csv('issue_body_after_remove.md')

time: 37.8 ms (started: 2023-05-02 13:31:02 -04:00)


In [60]:
for col in issues_body_clean.columns:
    print(col)

index
number
body
time: 704 µs (started: 2023-05-02 13:31:03 -04:00)


In [61]:
issues_body_clean.head()

,index,number,body
0,0,3695,At the moment any non essential commands are not being merged \r\nIf you want to add non essential commands to Auto GPT please create a plugin instead \r\nWe are expecting to ship plugin support within the week PR 757 \r\nResources \r...
1,1,3693,Search for existing issues first \n\n X I have searched the existing issues and there is no existing issue for my problem\n\n \n\nMacOS\n\n Which version of Auto GPT are you using \n\nStable branch \n\n GPT 3 or GPT 4 \n\nG...
2,2,3692,\r\n\r\n At the moment any non essential commands are not being merged \r\nIf you want to add non essential commands to Auto GPT please create a plugin instead \r\nWe are expecting to ship plugin support within the week PR 757 \r\nReso...
3,3,3690,Updating to encourage the user to build a virtual environment first \r\n\r\n At the moment any non essential commands are not being merged \r\nIf you want to add non essential commands to Auto GPT please create a plugin instead \r\nWe are...
4,4,3688,Background\r\n\r\nCurrently only ALLOWLISTED PLUGINS read from env \r\n\r\nAs more plugins are created you have to click n for each plugin you don t want to use enabling DENYLISTED PLUGINS will increase usability \r\n\r\nThere is anothe...


time: 8.58 ms (started: 2023-05-02 13:31:04 -04:00)


In [62]:
issues_body_clean.to_csv('my_data.md', index=False, columns=['body'])

time: 32.2 ms (started: 2023-05-02 13:31:05 -04:00)


In [63]:

# convert all non-string columns to string
issues_body_clean = issues_body_clean.astype(str)

# define a regular expression to match known characters and markdown syntax
regex_known = re.compile(r"[^\w\s.,!?;:\-'" r"\*\(\)\[\]#`_\{\}\+\-\\/]|_")

# apply the regular expression to each element of the DataFrame
issues_body_clean = issues_body_clean.applymap(lambda s: regex_known.sub(" ", s))

# function to extract phrases from a string
def extract_phrases(text, phrase_len):
    # split the text into words
    words = text.split()
    # initialize an empty list to store the phrases
    phrases = []
    # loop over the words, stopping at the last phrase_len words
    for i in range(len(words)-phrase_len+1):
        # extract the phrase of length phrase_len
        phrase = " ".join(words[i:i+phrase_len])
        # add the phrase to the list
        phrases.append(phrase)
    # return the list of phrases
    return phrases

# set the desired phrase length
phrase_len = 3

# initialize an empty dictionary to store the phrase counts
phrase_counts = {}

# loop over each row of the dataframe
for index, row in issues_body_clean.iterrows():
    # extract the text from the "body" column
    text = row["body"]
    # extract the phrases from the text
    phrases = extract_phrases(text, phrase_len)
    # loop over the phrases
    for phrase in phrases:
        # if the phrase is already in the dictionary, increment the count
        if phrase in phrase_counts:
            phrase_counts[phrase] += 1
        # otherwise, add the phrase to the dictionary with a count of 1
        else:
            phrase_counts[phrase] = 1

# create a dataframe from the phrase_counts dictionary
phrase_df = pd.DataFrame(list(phrase_counts.items()), columns=["phrase", "count"])

# sort the dataframe by count in descending order
phrase_df = phrase_df.sort_values("count", ascending=False)

# print the top 10 phrases
print(phrase_df.head(20))


                      phrase  count
310                 x I have    577
28909             2023 04 27    451
417                 X I have    417
28910               04 27 17    416
28911               27 17 43    416
28912               17 43 03    403
45721             2023 04 22    356
303            is atomic and    337
11651             2023 04 30    335
15721               04 30 06    331
103              on a single    323
421      the existing issues    315
420    searched the existing    315
419        have searched the    315
418          I have searched    315
9247       lib site packages    310
39          https github com    279
498      NEXT ACTION COMMAND    278
24580     r requirements txt    274
24581  requirements txt line    263
time: 290 ms (started: 2023-05-02 13:31:07 -04:00)


In [64]:
phrase_df.to_csv('phrase.csv', index=False)

time: 140 ms (started: 2023-05-02 13:31:12 -04:00)


In [51]:
counts_df.head

<bound method NDFrame.head of                                   count
hello world                           2
thanks in advance                     1
looking forward to your response      0>

time: 3.07 ms (started: 2023-05-02 12:41:11 -04:00)


In [ ]:
remove_part1 = ""<!-- 📢 Announcement
We've recently noticed an increase in pull requests focusing on combining multiple changes. While the intentions behind these PRs are appreciated, it's essential to maintain a clean and manageable git history. To ensure the quality of our repository, we kindly ask you to adhere to the following guidelines when submitting PRs:

Focus on a single, specific change.
Do not include any unrelated or ""extra"" modifications.
Provide clear documentation and explanations of the changes made.
Ensure diffs are limited to the intended lines — no applying preferred formatting styles or line endings (unless that's what the PR is about).
For guidance on committing only the specific lines you have changed, refer to this helpful video: https://youtu.be/8-hSNHHbiZg

By following these guidelines, your PRs are more likely to be merged quickly after testing, as long as they align with the project's overall direction. -->""

In [ ]:
issues_body.head()

In [ ]:
import re

# Extract the issue number from the body column and add it as a new column
#issues_body['Issue_number'] = issues_body['body'].str.extract(r'(\d+)')

# Remove the checkbox from the body column
issues_body['body'] = issues_body['body'].str.replace('- \[X\] ', '')

# Extract the section headers and their corresponding text into separate columns
headers = ['Steps to reproduce', 'Current behavior', 'Expected behavior', 'Your prompt']
for header in headers:
    issues_body[header] = issues_body['body'].str.extract(rf'{header}.*?\n(.*?)\n\n', flags=re.DOTALL | re.IGNORECASE)
    issues_body['body'] = issues_body['body'].str.replace(rf'{header}.*?\n(.*?)\n\n', '')

# Reorder the columns
issues_body = issues_body[['index', 'number', 'Steps to reproduce', 'Current behavior', 'Expected behavior', 'Your prompt', 'body']]


In [ ]:
issues_body.head()